In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
stopword = nltk.corpus.stopwords.words('english')
wn=nltk.WordNetLemmatizer()
st=nltk.PorterStemmer()

In [3]:
def clean(text):
    s = re.sub(r'Description ', '',text) 
    s = re.sub(r' show more', '',s).strip().lower()
    text=re.sub(r'[^\w\s]',' ',s).lower()   ### Remove punctuation by removing non-space alpha-num characters
    words=re.split(r'\W+', text) ### Split by remaining non-alphanumeric characters
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    lem_words= [wn.lemmatize(word) for word in non_stop_words]             ### Clean text ==
    clean_text= ' '.join(lem_words)
    return clean_text

In [4]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
files = []
for infile in sorted(glob.glob('*_info.txt'), key=numericalSort):
    files.append(infile)

In [10]:
files

['1_info.txt',
 '2_info.txt',
 '3_info.txt',
 '4_info.txt',
 '5_info.txt',
 '6_info.txt',
 '7_info.txt',
 '8_info.txt',
 '9_info.txt',
 '10_info.txt',
 '11_info.txt',
 '12_info.txt',
 '13_info.txt',
 '14_info.txt',
 '15_info.txt',
 '16_info.txt',
 '17_info.txt',
 '18_info.txt',
 '19_info.txt',
 '20_info.txt',
 '21_info.txt',
 '22_info.txt',
 '23_info.txt',
 '24_info.txt',
 '25_info.txt',
 '26_info.txt',
 '27_info.txt',
 '28_info.txt',
 '29_info.txt',
 '30_info.txt',
 '31_info.txt',
 '32_info.txt',
 '33_info.txt',
 '34_info.txt',
 '35_info.txt',
 '36_info.txt',
 '37_info.txt',
 '38_info.txt',
 '39_info.txt',
 '40_info.txt',
 '41_info.txt',
 '42_info.txt',
 '43_info.txt',
 '44_info.txt',
 '45_info.txt',
 '46_info.txt',
 '47_info.txt',
 '48_info.txt',
 '49_info.txt',
 '50_info.txt',
 '51_info.txt',
 '52_info.txt',
 '53_info.txt',
 '54_info.txt',
 '55_info.txt',
 '56_info.txt',
 '57_info.txt',
 '58_info.txt',
 '59_info.txt',
 '60_info.txt',
 '61_info.txt',
 '62_info.txt',
 '63_info.txt',
 

In [9]:
df = pd.DataFrame()
count = 0
while count < 1999:
    for file in files:
        with open(file,'r',encoding="utf8") as f:
            data=f.read().split('\n\n')
            data=[i.split('\n') for i in data]
            s=pd.DataFrame([data[0][0:15]],columns=['id', 'title', 'purpose', 'total_enrollment', 'rating', 'number_ratings', 'last_update_date', 'what_you_get', 'prerequisites', 'description', 'five_stars', 'four_stars', 'three_stars', 'two_stars', 'one_star'])
            df = s
            df =pd.concat([df, s], ignore_index=True)
            count += 1

#df = df.iloc[1: , :]
#df.set_index('id', inplace=True)

ValueError: 15 columns passed, passed data had 2 columns

In [ ]:
df['description_clean'] = df['description'].apply(clean)

In [ ]:
df2 = pd.read_csv('C:/Users/15512/course_list_all.tsv', sep='\t')

In [240]:
complete_data = pd.concat([df, df2[:2001]], axis = 'columns')

In [243]:
complete_data.shape

(427, 26)

In [244]:
complete_data.columns

Index(['id', 'title', 'purpose', 'total_enrollment', 'rating',
       'number_ratings', 'last_update_date', 'what_you_get', 'prerequisites',
       'description', 'five_stars', 'four_stars', 'three_stars', 'two_stars',
       'one_star', 'description_clean', 'id', 'name', 'link', 'seller',
       'price', 'original_price', 'lectures', 'hours', 'level',
       'Tue Dec 14 05:56:01 2021'],
      dtype='object')

In [251]:
complete_data.drop(complete_data.columns[[4, 5, 16, 17, 25]], axis = 1, inplace=True)

In [ ]:
tfidf_trans=TfidfVectorizer()
X_tfidf=tfidf_trans.fit_transform(complete_data['description_clean'])

In [255]:
def Recommend():
    input_string = input('Enter the description of Udemy Course:')
    input_string_clean = clean(input_string)
    input_string_v = tfidf_trans.transform([input_string_clean])
    cosine_simil = cosine_similarity(input_string_v, X_tfidf)[0]
    sorted_index= np.argsort(cosine_simil)[::-1]
    return pd.DataFrame([complete_data.iloc[i].to_numpy().T for i in sorted_index], columns = complete_data.columns).head()

In [256]:
Recommend()

Enter the description of Udemy Course:You've definitely heard of AI and Deep Learning. But when you ask yourself, what is my position with respect to this new industrial revolution, that might lead you to another fundamental question: am I a consumer or a creator? For most people nowadays, the answer would be, a consumer.  But what if you could also become a creator?  What if there was a way for you to easily break into the World of Artificial Intelligence and build amazing applications which leverage the latest technology to make the World a better place?  Sounds too good to be true, doesn't it?  But there actually is a way..  Computer Vision is by far the easiest way of becoming a creator.  And it's not only the easiest way, it's also the branch of AI where there is the most to create.  Why? You'll ask.  That's because Computer Vision is applied everywhere. From health to retail to entertainment - the list goes on. Computer Vision is already a $18 Billion market and is growing expone

,title,purpose,total_enrollment,last_update_date,what_you_get,prerequisites,description,five_stars,four_stars,three_stars,two_stars,one_star,description_clean,link,seller,price,original_price,lectures,hours,level
0,Deep Learning and Computer Vision A Z OpenCV...,Become a Wizard of all the latest Computer Vis...,43258,12/2021,Have a toolbox of the most powerful Computer V...,Requirements Only High School Maths Basic Pyth...,Description *** AS SEEN ON KICKSTARTER *** You...,53%,32%,10%,3%,2%,seen kickstarter definitely heard ai deep lea...,https://www.udemy.com/course/computer-vision-a-z/,Hadelin de Ponteves Kirill Eremenko Ligency ...,13.99,84.99,85,11,All Levels
1,Computer Vision Masterclass,Learn in practice everything you need to know ...,26710,10/2021,Understand the basic intuition about Cascade a...,Requirements Programming logic Basic Python pr...,Description Computer Vision is a subarea of Ar...,51%,33%,13%,2%,1%,computer vision subarea artificial intelligenc...,https://www.udemy.com/course/computer-vision-m...,Jones Granatyr Ligency I Team Gabriel Alves,13.99,84.99,228,25.5,All Levels
2,Computer Vision In Python Face Detection Im...,Learn Computer Vision With OpenCV In Python! M...,16702,8/2021,Use OpenCV to work with image files Understand...,Requirements Basic Python programming knowledge,Description Computer vision is an interdiscipl...,46%,39%,11%,3%,1%,computer vision interdisciplinary field deal c...,https://www.udemy.com/course/computer-vision-i...,Emenwa Global Zoolord Academy,13.99,84.99,61,11,All Levels
3,Computer Vision Face Recognition Quick Starte...,Quickly Build Python Deep Learning based Face ...,3229,12/2021,Face Detection from Images Face Detection from...,Requirements A decent configuration computer a...,Description Hi There! welcome to my new cours...,45%,42%,10%,2%,1%,hi welcome new course face recognition deep le...,https://www.udemy.com/course/computer-vision-f...,Abhilash Nelson,15.99,99.99,64,6.5,All Levels
4,Automated Multiple Face Recognition AI Using P...,"Learn about OpenCv Basics, Face Recognition in...",19677,11/2019,Automated Multiple Face Recognition in an imag...,Requirements Basics of Python Programming,"Description Hello, welcome to the Amazing worl...",27%,31%,27%,9%,6%,hello welcome amazing world computer vision co...,https://www.udemy.com/course/automated-multipl...,Nishit Maru Three Millennials,13.99,84.99,10,2,All Levels


In [205]:
df2 = pd.read_csv('C:/Users/15512/course_list_all.tsv', sep='\t')

In [206]:
df2

,id,name,link,seller,price,original_price,lectures,hours,level,Tue Dec 14 05:56:01 2021
0,1,machinelearning/,https://www.udemy.com/course/machinelearning/,Kirill Eremenko Hadelin de Ponteves Ligency ...,13.99,84.99,320,44.5,All Levels,NaN
1,2,python-for-data-science-and-machine-learning-b...,https://www.udemy.com/course/python-for-data-s...,Jose Portilla,13.99,84.99,165,25,All Levels,NaN
2,3,the-data-science-course-complete-data-science-...,https://www.udemy.com/course/the-data-science-...,365 Careers 365 Careers Team,15.99,94.99,488,30,All Levels,NaN
3,4,r-programming/,https://www.udemy.com/course/r-programming/,Kirill Eremenko Ligency I Team Ligency Team,13.99,84.99,82,10.5,All Levels,NaN
4,5,deeplearning/,https://www.udemy.com/course/deeplearning/,Kirill Eremenko Hadelin de Ponteves Ligency ...,13.99,84.99,173,22.5,All Levels,NaN
...,...,...,...,...,...,...,...,...,...,...
2187,2188,master-en-machine-learning/,https://www.udemy.com/course/master-en-machine...,Juan Gabriel Gomila Salas Mar a Santos Arnau...,Free,NaN,73,16.5,Beginner,NaN
2188,2189,data-science-foundations/,https://www.udemy.com/course/data-science-foun...,Eng Mustafa Othman,Free,NaN,11,2,Beginner,NaN
2189,2190,introduccion-gratuita-python-data-science/,https://www.udemy.com/course/introduccion-grat...,Rafael Gonzalez Gouveia,Free,NaN,13,1 total,Beginner,NaN
2190,2191,guia-iniciantes-machine-learning-data-science/,https://www.udemy.com/course/guia-iniciantes-m...,Jones Granatyr IA Expert Academy,Free,NaN,28,2,Beginner,NaN


In [226]:
complete_df123456 = pd.concat([df, df2[:427]], axis = 'columns')

In [227]:
complete_df123456

,id,title,purpose,total_enrollment,rating,number_ratings,last_update_date,what_you_get,prerequisites,description,...,id,name,link,seller,price,original_price,lectures,hours,level,Tue Dec 14 05:56:01 2021
0,1,Machine Learning A Z Hands On Python R In ...,Learn to create Machine Learning Algorithms in...,833575,NULL,NULL,12/2021,Master Machine Learning on Python R Have a g...,Requirements Just some high school mathematics...,Description Interested in the field of Machine...,...,1,machinelearning/,https://www.udemy.com/course/machinelearning/,Kirill Eremenko Hadelin de Ponteves Ligency ...,13.99,84.99,320,44.5,All Levels,NaN
1,2,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",501618,NULL,NULL,5/2020,Use Python for Data Science and Machine Learni...,Requirements Some programming experience Admin...,Description Are you ready to start your path t...,...,2,python-for-data-science-and-machine-learning-b...,https://www.udemy.com/course/python-for-data-s...,Jose Portilla,13.99,84.99,165,25,All Levels,NaN
2,3,The Data Science Course 2021 Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",452453,NULL,NULL,11/2021,The course provides the entire toolbox you nee...,Requirements No prior experience is required ...,Description The Problem Data scientist is one ...,...,3,the-data-science-course-complete-data-science-...,https://www.udemy.com/course/the-data-science-...,365 Careers 365 Careers Team,15.99,94.99,488,30,All Levels,NaN
3,4,R Programming A Z R For Data Science With Re...,Learn Programming In R And R Studio. Data Anal...,221022,NULL,NULL,12/2021,Learn to program in R at a good level Learn ho...,Requirements,Description Learn R Programming by doing! Ther...,...,4,r-programming/,https://www.udemy.com/course/r-programming/,Kirill Eremenko Ligency I Team Ligency Team,13.99,84.99,82,10.5,All Levels,NaN
4,5,Deep Learning A Z Hands On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,322765,NULL,NULL,12/2021,Understand the intuition behind Artificial Neu...,Requirements,Description *** As seen on Kickstarter *** Art...,...,5,deeplearning/,https://www.udemy.com/course/deeplearning/,Kirill Eremenko Hadelin de Ponteves Ligency ...,13.99,84.99,173,22.5,All Levels,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,423,Time Series Analysis and Forecasting with Python,"Learn Python for Pandas, Statsmodels, ARIMA, S...",3492,NULL,NULL,8/2021,Basic Packages NumPy Pandas Matplotlib Tim...,Requirements General and Basic Python Skills,"Description ""Time Series Analysis and Forecast...",...,423,time-series-analysis-and-forecasting-with-python/,https://www.udemy.com/course/time-series-analy...,Navid Shirzadi,13.99,84.99,51,10.5,All Levels,NaN
423,424,Artificial Intelligence in Arabic ...,تعلم مبادئ الذكاء الصناعي عن طريق ١٠ مشاريع عملية,998,NULL,NULL,2/2019,Numpy pandas matplotlib seaborn ...,Requirements,Description كورس كامل لتعليم مبادئ الذكاء الصن...,...,424,artificial-intelligence-in-arabic/,https://www.udemy.com/course/artificial-intell...,Dr Ryan Ahmed Ph D MBA Mitchell Bouchard ...,13.99,84.99,57,10,All Levels,NaN
424,425,Natural Language Processing NLP In Python wit...,"Learn NLP with Machine Learning Algorithms, Sp...",19672,NULL,NULL,7/2021,Get to learn about different Applications of N...,Requirements,Description Interested in Learning Natural Lan...,...,425,nlp-bootcamp-machine-learning-deep-learning/,https://www.udemy.com/course/nlp-bootcamp-mach...,Data Is Good Academy,13.99,19.99,62,3,Beginner,NaN
425,426,100 Exercises Python Programming Data Sci...,Improve your Python programming and data scien...,40347,NULL,NULL,12/2021,solve over 100 exercises in NumPy deal with re...,Requirements completed course 200 Exercises ...,Description ----------------------------------...,...,426,100-exercises-python-programming-data-science-...,https://www.udemy.com/course/100-exercises-pyt...,Pawe Krakowiak takeITeasy A